In [1]:
import glob
from time import perf_counter as clock
from os import path

from astropy.io import fits
from astropy.table import Table
from astropy import modeling
import numpy as np

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util

import pandas as pd 
from tqdm import tqdm
import matplotlib.pyplot as plt
from shutil import copyfile
from IPython.display import clear_output
import os

In [2]:
#!/usr/bin/env python
##############################################################################
#
# Usage example for the procedure PPXF, which implements the
# Penalized Pixel-Fitting (pPXF) method originally described in
# Cappellari M., & Emsellem E., 2004, PASP, 116, 138
#     http://adsabs.harvard.edu/abs/2004PASP..116..138C
# and upgraded in Cappellari M., 2017, MNRAS, 466, 798
#     http://adsabs.harvard.edu/abs/2017MNRAS.466..798C
#
# The example also shows how to include a library of templates
# and how to mask gas emission lines if present.
# The example is specialized for a fit to a SDSS spectrum.
#
# MODIFICATION HISTORY:
#   V1.0.0: Written by Michele Cappellari, Leiden 11 November 2003
#   V1.1.0: Log rebin the galaxy spectrum. Show how to correct the velocity
#       for the difference in starting wavelength of galaxy and templates.
#       MC, Vicenza, 28 December 2004
#   V1.1.1: Included explanation of correction for instrumental resolution.
#       After feedback from David Valls-Gabaud. MC, Venezia, 27 June 2005
#   V2.0.0: Included example routine to determine the goodPixels vector
#       by masking known gas emission lines. MC, Oxford, 30 October 2008
#   V2.0.1: Included instructions for high-redshift usage. Thanks to Paul Westoby
#       for useful feedback on this issue. MC, Oxford, 27 November 2008
#   V2.0.2: Included example for obtaining the best-fitting redshift.
#       MC, Oxford, 14 April 2009
#   V2.1.0: Bug fix: Force PSF_GAUSSIAN to produce a Gaussian with an odd
#       number of elements centered on the middle one. Many thanks to
#       Harald Kuntschner, Eric Emsellem, Anne-Marie Weijmans and
#       Richard McDermid for reporting problems with small offsets
#       in systemic velocity. MC, Oxford, 15 February 2010
#   V2.1.1: Added normalization of galaxy spectrum to avoid numerical
#       instabilities. After feedback from Andrea Cardullo.
#       MC, Oxford, 17 March 2010
#   V2.2.0: Perform templates convolution in linear wavelength.
#       This is useful for spectra with large wavelength range.
#       MC, Oxford, 25 March 2010
#   V2.2.1: Updated for Coyote Graphics. MC, Oxford, 11 October 2011
#   V2.3.0: Specialized for SDSS spectrum following requests from users.
#       Renamed PPXF_KINEMATICS_EXAMPLE_SDSS. MC, Oxford, 12 January 2012
#   V3.0.0: Translated from IDL into Python. MC, Oxford, 10 December 2013
#   V3.0.1: Uses MILES models library. MC, Oxford 11 December 2013
#   V3.0.2: Support both Python 2.6/2.7 and Python 3.x. MC, Oxford, 25 May 2014
#   V3.0.3: Explicitly sort template files as glob() output may not be sorted.
#       Thanks to Marina Trevisan for reporting problems under Linux.
#       MC, Sydney, 4 February 2015
#   V3.0.4: Use redshift in determine_goodpixels. MC, Oxford, 5 May 2015
#   V3.1.0: Illustrate how to deal with variable instrumental resolution.
#       Use example galaxy spectrum from SDSS DR12. MC, Oxford, 12 October 2015
#   V3.1.1: Support both Pyfits and Astropy to read FITS files.
#       MC, Oxford, 22 October 2015
#   V3.1.2: Illustrates how to show the wavelength in the plot.
#       MC, Oxford, 18 May 2016
#   V3.1.3: Make files paths relative to this file, to run the example from
#       any directory. MC, Oxford, 18 January 2017
#   V3.1.4: Updated text on the de-redshifting of the spectrum.
#       MC, Oxford, 5 October 2017
#   V3.1.5: Changed imports for pPXF as a package.
#       Make file paths relative to the pPXF package to be able to run the
#       example unchanged from any directory. MC, Oxford, 17 April 2018
#   V3.1.6: Dropped legacy Python 2.7 support. MC, Oxford, 10 May 2018
#   V3.1.7: Fixed clock DeprecationWarning in Python 3.7.
#       MC, Oxford, 27 September 2018
#
##############################################################################

def ppxf_example_kinematics_sdss(PATH,name,cl={},cl_continuum=[]):

    ppxf_dir = path.dirname(path.realpath(ppxf_package.__file__))

    # Read SDSS DR12 galaxy spectrum taken from here http://dr12.sdss3.org/
    # The spectrum is *already* log rebinned by the SDSS DR12
    # pipeline and log_rebin should not be used in this case.
    #file = ppxf_dir + '/spectra/NGC4636_SDSS_DR12.fits'
    file=PATH+name
    hdul = fits.open(file)
    t = hdul[1].data
    #t_2 = hdul[2].data  
    z = hdul[2].data.Z # SDSS redshift estimate

    # Only use the wavelength range in common between galaxy and stellar library.
    mask = (t['loglam'] > np.log10(3540*(1+z))) & (t['loglam'] < np.log10(7409*(1+z)))
    flux = t['flux'][mask]
    galaxy = flux/np.median(flux)   # Normalize spectrum to avoid numerical issues
    loglam_gal = t['loglam'][mask]
    lam_gal = 10**loglam_gal
    noise = np.full_like(galaxy, 0.0166)       # Assume constant noise per pixel here

    c = 299792.458                  # speed of light in km/s
    frac = lam_gal[1]/lam_gal[0]    # Constant lambda fraction per pixel
    dlam_gal = (frac - 1)*lam_gal   # Size of every pixel in Angstrom
    wdisp = t['wdisp'][mask]        # Intrinsic dispersion of every pixel, in pixels units
    fwhm_gal = 2.355*wdisp*dlam_gal # Resolution FWHM of every pixel, in Angstroms
    velscale = np.log(frac)*c       # Velocity scale in km/s per pixel (eq.8 of Cappellari 2017)

    # If the galaxy is at significant redshift, one should bring the galaxy
    # spectrum roughly to the rest-frame wavelength, before calling pPXF
    # (See Sec.2.4 of Cappellari 2017). In practice there is no
    # need to modify the spectrum in any way, given that a red shift
    # corresponds to a linear shift of the log-rebinned spectrum.
    # One just needs to compute the wavelength range in the rest-frame
    # and adjust the instrumental resolution of the galaxy observations.
    # This is done with the following three commented lines:
    #
    #lam_gal = lam_gal/(1+z)     # Compute approximate restframe wavelength
    #fwhm_gal = fwhm_gal/(1+z)   # Adjust resolution in Angstrom
    #z = 0                       # Spectrum is now in rest-frame

    # Read the list of filenames from the Single Stellar Population library
    # by Vazdekis (2010, MNRAS, 404, 1639) http://miles.iac.es/. A subset
    # of the library is included for this example with permission
    vazdekis = glob.glob(ppxf_dir + '/miles_models/Mun1.30Z*.fits')
    fwhm_tem = 2.51 # Vazdekis+10 spectra have a constant resolution FWHM of 2.51A.

    # Extract the wavelength range and logarithmically rebin one spectrum
    # to the same velocity scale of the SDSS galaxy spectrum, to determine
    # the size needed for the array which will contain the template spectra.
    #
    hdu = fits.open(vazdekis[0])
    ssp = hdu[0].data
    h2 = hdu[0].header
    lam_temp = h2['CRVAL1'] + h2['CDELT1']*np.arange(h2['NAXIS1'])
    lamRange_temp = [np.min(lam_temp), np.max(lam_temp)]
    sspNew = util.log_rebin(lamRange_temp, ssp, velscale=velscale)[0]
    templates = np.empty((sspNew.size, len(vazdekis)))

    # Interpolates the galaxy spectral resolution at the location of every pixel
    # of the templates. Outside the range of the galaxy spectrum the resolution
    # will be extrapolated, but this is irrelevant as those pixels cannot be
    # used in the fit anyway.
    fwhm_gal = np.interp(lam_temp, lam_gal, fwhm_gal)

    # Convolve the whole Vazdekis library of spectral templates
    # with the quadratic difference between the SDSS and the
    # Vazdekis instrumental resolution. Logarithmically rebin
    # and store each template as a column in the array TEMPLATES.

    # Quadratic sigma difference in pixels Vazdekis --> SDSS
    # The formula below is rigorously valid if the shapes of the
    # instrumental spectral profiles are well approximated by Gaussians.
    #
    # In the line below, the fwhm_dif is set to zero when fwhm_gal < fwhm_tem.
    # In principle it should never happen and a higher resolution template should be used.
    #
    fwhm_dif = np.sqrt((fwhm_gal**2 - fwhm_tem**2).clip(0))
    sigma = fwhm_dif/2.355/h2['CDELT1'] # Sigma difference in pixels

    for j, fname in enumerate(vazdekis):
        hdu = fits.open(fname)
        ssp = hdu[0].data
        ssp = util.gaussian_filter1d(ssp, sigma)  # perform convolution with variable sigma
        sspNew = util.log_rebin(lamRange_temp, ssp, velscale=velscale)[0]
        templates[:, j] = sspNew/np.median(sspNew) # Normalizes templates

    # The galaxy and the template spectra do not have the same starting wavelength.
    # For this reason an extra velocity shift DV has to be applied to the template
    # to fit the galaxy spectrum. We remove this artificial shift by using the
    # keyword VSYST in the call to PPXF below, so that all velocities are
    # measured with respect to DV. This assume the redshift is negligible.
    # In the case of a high-redshift galaxy one should de-redshift its
    # wavelength to the rest frame before using the line below (see above).
    #
    c = 299792.458   # km/s
    dv = c*np.log(lam_temp[0]/lam_gal[0])    # eq.(8) of Cappellari (2017)
    if (chunk.SUBCLASS[i] in ['AGN BROADLINE','STARBURST BROADLINE','STARFORIMNG BROADLINE','BROADLINE']):
        W=5000
    else: W=800
    goodpixels = util.determine_goodpixels(np.log(lam_gal), lamRange_temp, z, W)

    # Here the actual fit starts. The best fit is plotted on the screen.
    # Gas emission lines are excluded from the pPXF fit using the GOODPIXELS keyword.
    #
    vel = c*np.log(1 + z)   # eq.(8) of Cappellari (2017)
    start = [vel, 200.]  # (km/s), starting guess for [V, sigma]
    t = clock()
   
    if len(templates) < len(galaxy):
        galaxy=galaxy[0:len(templates)]
        noise=noise[0:len(templates)]
        lam_gal=lam_gal[0:len(templates)]
    pp = ppxf(templates, galaxy, noise, velscale, start,
              goodpixels=goodpixels, plot=False, moments=4,
              degree=12, vsyst=dv, clean=False, lam=lam_gal)

    print("Formal errors:")
    print("     dV    dsigma   dh3      dh4")
    print("".join("%8.2g" % f for f in pp.error*np.sqrt(pp.chi2)))

    print('Elapsed time in PPXF: %.2f s' % (clock() - t))

    # If the galaxy is at significant redshift z and the wavelength has been
    # de-redshifted with the three lines "z = 1.23..." near the beginning of
    # this procedure, the best-fitting redshift is now given by the following
    # commented line (equation 2 of Cappellari et al. 2009, ApJ, 704, L34):
    #
   # print ('Best-fitting redshift z:', (hdul[2].data.Z + 1)*(1 + pp.sol[0]/c) - 1)
    data=pp.bestfit
    
    fig=plt.subplots(nrows=3,ncols=5,facecolor='white',dpi=300,figsize=(14,10))
    ax1=plt.subplot(3,5,(1,5))
    ax1.set_title('GroupID=%s, Groupsize=%s, specname=%s \n z=%s, BPT=%s, SUBCLASS=%s, CL_COUNTS=%s \n V=%f, sigma=%f, h3=%f, h4=%f'%(chunk.GroupID[i],chunk.GroupSize[i],\
                            name,hdul[2].data.Z,chunk.BPT_PORTSMOUTH[i],chunk.SUBCLASS[i],chunk.CL_COUNTS_UMAP[i],\
                            pp.sol[0],pp.sol[1],pp.sol[2],pp.sol[3]))                                                                                       
    pp.plot(Z=z)
        
    
    #fit and plot the coronal lines,calculate the EW of coronal lines
    zz=6
    lam_gal_correct=lam_gal/(10*(1+z))
    resi=galaxy-data
    for coro in cl:
        
        line_co=cl[coro]
        ind_0=np.where((((lam_gal_correct) > line_co-0.5) & ((lam_gal_correct) < line_co+0.5)),True,False)
        ind_1=np.where(((lam_gal_correct) > cl_continuum[zz-6][0]) & ((lam_gal_correct) < cl_continuum[zz-6][1]))
        ind_2=np.where(((lam_gal_correct>line_co-2.5) & (lam_gal_correct<line_co+2.5)),True,False)
        
        
        if (line_co > np.min(lam_gal_correct)) and (line_co < np.max(lam_gal_correct)):
            ind_3=np.where((((lam_gal_correct) > line_co-10) & ((lam_gal_correct) < line_co+10)),True,False)
            center=np.max(resi[ind_0])
            #line_center=(lam_gal_correct==line_co)           
            MEAN=lam_gal_correct[ind_0][np.where(resi[ind_0]==center)]
            Am=center
            g_init = modeling.models.Gaussian1D(amplitude=Am, mean=MEAN, stddev=2.)
            fit_g = modeling.fitting.LevMarLSQFitter()
            g = fit_g(g_init,lam_gal_correct[ind_2],resi[ind_2])
            g_plot=g(lam_gal_correct[ind_3])
            '''
            MEAN_l=lam_gal_correct[line_center]
            Am_l=resi[line_center]
            g_init_l = modeling.models.Gaussian1D(amplitude=Am_l,mean=MEAN_l,stddev=5.)
            fit_g_l = modeling.fitting.LevMarLSQFitter()
            g_l = fit_g_l(g_init_l,lam_gal_correct,resi)'''
            
            
        else:
            ind_3=np.where((((lam_gal_correct) > np.min(lam_gal_correct)) & ((lam_gal_correct) < np.max(lam_gal_correct))),True,False)
            g_plot=np.full_like(lam_gal_correct[ind_3],-999)
        
        ax2=plt.subplot(3,5,zz)
        ax2.set_ylim(-0.5,2)
        ax2.set_xlim(float(line_co)-10,float(line_co)+10)
        ax2.plot(lam_gal_correct,galaxy,color='black')
        ax2.plot(lam_gal_correct,data,color='red')
        ax2.plot(lam_gal_correct,galaxy-data,'.',color='green',alpha=0.6)
        ax2.plot(lam_gal_correct[ind_3],g_plot,color='orange')
        #ax2.plot(lam_gal_correct,g_l(lam_gal_correct),color='lightblue')
        ax2.axvline(x=line_co,ls='--')
        ax2.text(line_co,0.6,r'$%s$'%coro,fontsize=6,rotation=90,color='blue')
        #print('-------------------------------------------')
        #print('%s:   '%coro,np.sum(ew[ind_0]))
        
        ew_diff=((galaxy-data)/data)*np.median(flux)
        rms=np.sqrt(np.sum(resi[ind_1]**2)/len(resi[ind_1]))
             
        chunk.loc[chunk.specname==name,'EW_%s'%coro]=np.sum(ew_diff[ind_0])
        chunk.loc[chunk.specname==name,'normEW_%s'%coro]=np.sum(((galaxy-data)/data)*[ind_0])
        chunk.loc[chunk.specname==name,'normLINESIGMA_%s'%coro]=Am/rms
        
        zz+=1
        
    chunk.loc[chunk.specname==name,'vel']=pp.sol[0]
    chunk.loc[chunk.specname==name,'vel_sigma']=pp.sol[1]
    
    #write to fits files
    if os.path.exists('/media/richard/Backup Plus/coronal_line_sample_repeat_0810/fitting_spectra/Fitting_%s'%name)==False:
        
        hdr=fits.Header()
        hdr['velocity']=pp.sol[0]
        hdr['vel_sigma']=pp.sol[1]
        st=0
        for string in ['dv','dsigma','h3','h4']:
            hdr['%s'%string]=(pp.error*np.sqrt(pp.chi2))[st]
            empty_primary = fits.PrimaryHDU(header=hdr)
            st+=1
        c1 = fits.Column(name='lam', array=lam_gal, format='K')
        c2 = fits.Column(name='norm_flux', array=galaxy, format='K')
        c3 = fits.Column(name='bestfit_flux', array=data, format='K')
        c4 = fits.Column(name='residual_flux', array=galaxy-data, format='K')
        table_hdu = fits.BinTableHDU.from_columns([c1, c2, c3, c4])
        hdul=fits.HDUList([empty_primary,table_hdu])       
        hdul.writeto('/media/richard/Backup Plus/coronal_line_sample_repeat_0810/fitting_spectra/Fitting_%s'%name)
    
    else:
        with fits.open('/media/richard/Backup Plus/coronal_line_sample_repeat_0810/fitting_spectra/Fitting_%s'%name) as hdu:
            dat=hdu[1].data
            hdr=hdu[0].header
        
            dat['lam']=lam_gal
            dat['norm_flux']=galaxy
            dat['bestfit_flux']=data
            dat['residual_flux']=galaxy-data
        
            hdr['velocity']=pp.sol[0]
            hdr['vel_sigma']=pp.sol[1]
            st=0
            for string in ['dv','dsigma','h3','h4']:
                hdr['%s'%string]=(pp.error*np.sqrt(pp.chi2))[st]
                empty_primary = fits.PrimaryHDU(header=hdr)
                st+=1
            hdu.flush()
    
    plt.savefig('/media/richard/Backup Plus/coronal_line_sample_repeat_0810/fitting/%s.png'%name)
    #copyfile(file,'/media/richard/Backup Plus/coronal_line_sample_repeat/spectra/%s'%name)
    #np.savetxt('/media/richard/Backup Plus/coronal_line_sample_repeat/fitting_doc/ppxf_%s_error.txt',pp.error*np.sqrt(pp.chi2))
    

#------------------------------------------------------------------------------

In [3]:
chunks=[]
if __name__ == '__main__':
    #for j in range(0,2529,200):
    #if j>0:
    #data=pd.read_csv('/media/richard/Backup Plus/314kpair_DR16_final_Z_0.4_v13_CL_2529_822.csv')
    #df=pd.read_csv('/media/richard/Backup Plus/314kpair_DR16_final_Z_0.4_v13_CL_2529_822.csv',chunksize=100,low_memory=False,skiprows=j,nrows=200,names=data.columns)
    #else:
    df=pd.read_csv('/media/richard/Backup Plus/coronal_line_sample_repeat_0810/314kpair_DR16_final_Z_0.4_v13_104_recycle_393.csv',chunksize=300,low_memory=False)
    PATH='/media/richard/Backup Plus/sdss_16_pair/'
    CL={'FeVII]3759':375.89,'ArXIV]4414':441.25,'FeVII]5160':515.90,'FeXIV]5304':530.29,'FeVI]5335':533.52,\
        'FeVI]5426':542.66,'Fe VII]5722':572.07,'FeVII]6087':608.70,'FeX]6375':637.45,'ArV]7006':700.58}
    CL_CON=[[377.5,383.2],[420.0,423.0],[520.0,526.0],[520.0,526.0],[520.0,526.0],[547.5,553.5],[547.5,553.5],\
            [611.0,620.0],[610.0,620.0],[680.0,686.0]]
    for chunk in tqdm(df):
        for i in chunk.index:
            name=chunk.specname[i]
            try:
                ppxf_example_kinematics_sdss(PATH,name,cl=CL,cl_continuum=CL_CON)
                plt.pause(1)
                plt.close()
                print(i)
            except:
                pass
        chunks.append(chunk)
        clear_output()
        print('Fitting succeeded with 300 spectra')
    df2=pd.concat(chunks)
    df2=df2.replace([np.inf, -np.inf],-999)
    df2=df2.fillna(-999)
    df2.to_csv('/media/richard/Backup Plus/coronal_line_sample_repeat_0810/314kpair_DR16_final_Z_0.4_v13_104_recycle_393.csv')

2it [49:38, 1489.02s/it]

Fitting succeeded with 300 spectra


In [26]:
d=pd.read_csv('/media/richard/Backup Plus/coronal_line_sample_repeat_0810/314kpair_DR16_final_Z_0.4_cl_1410_5sigma_395_visual_220.csv')
df2=pd.read_csv('/media/richard/Backup Plus/coronal_line_sample_repeat_0810/314kpair_DR16_final_Z_0.4_v13_104_recycle_393.csv')
CL={'FeVII]3759':375.89,'ArXIV]4414':441.25,'FeVII]5160':515.90,'FeXIV]5304':530.29,'FeVI]5335':533.52,\
        'FeVI]5426':542.66,'Fe VII]5722':572.07,'FeVII]6087':608.70,'FeX]6375':637.45,'ArV]7006':700.58}
d=d.replace([np.inf, -np.inf],-999)
d=d.fillna(-999)
for i in tqdm(df2.index):
    cut=df2[df2.GroupID==df2.GroupID[i]]
    for x in CL:
        change=(max(cut['EW_%s'%x])-min(cut['EW_%s'%x]))/max(cut['EW_%s'%x])
        df2.loc[df2.GroupID==df2.GroupID[i],'EW_%s_ratio'%x]=change
    i+=len(cut)

for i in range(len(d)):
    if d.visual_check[i]==1:
        df2.visual_check[df2.specname==d.specname[i]]=2

df2.visual_check[df2.visual_check==1]=0
df2.visual_check[df2.visual_check==2]=1
df2.visual_check

df2.to_csv('/media/richard/Backup Plus/coronal_line_sample_repeat_0810/314kpair_DR16_final_Z_0.4_v13_104_recycle_393.csv')

100%|██████████| 393/393 [00:01<00:00, 317.64it/s]


In [ ]:
#!/usr/bin/env python
##############################################################################
#
# Usage example for the procedure PPXF, which implements the
# Penalized Pixel-Fitting (pPXF) method originally described in
# Cappellari M., & Emsellem E., 2004, PASP, 116, 138
#     http://adsabs.harvard.edu/abs/2004PASP..116..138C
# and upgraded in Cappellari M., 2017, MNRAS, 466, 798
#     http://adsabs.harvard.edu/abs/2017MNRAS.466..798C
#
# This example shows how to study stellar population and include gas emission
# lines as templates instead of masking them using the GOODPIXELS keyword.
#
# MODIFICATION HISTORY:
#   V1.0.0: Adapted from PPXF_KINEMATICS_EXAMPLE.
#       Michele Cappellari, Oxford, 12 October 2011
#   V1.1.0: Made a separate routine for the construction of the templates
#       spectral library. MC, Vicenza, 11 October 2012
#   V1.1.1: Includes regul_error definition. MC, Oxford, 15 November 2012
#   V2.0.0: Translated from IDL into Python. MC, Oxford, 6 December 2013
#   V2.0.1: Fit SDSS rather than SAURON spectrum. MC, Oxford, 11 December 2013
#   V2.1.0: Includes gas emission as templates instead of masking the spectrum.
#       MC, Oxford, 7 January 2014
#   V2.1.1: Support both Python 2.6/2.7 and Python 3.x. MC, Oxford, 25 May 2014
#   V2.1.2: Illustrates how to print and plot emission lines.
#       MC, Oxford, 5 August 2014
#   V2.1.3: Only includes emission lines falling within the fitted wavelength
#       range. MC, Oxford, 3 September 2014
#   V2.1.4: Explicitly sort template files as glob() output may not be sorted.
#       Thanks to Marina Trevisan for reporting problems under Linux.
#       MC, Sydney, 4 February 2015
#   V2.1.5: Included origin='upper' in imshow(). Thanks to Richard McDermid
#       for reporting a different default value with older Matplotlib versions.
#       MC, Oxford, 17 February 2015
#   V2.1.6: Use color= instead of c= to avoid new Matplotlib bug.
#       MC, Oxford, 25 February 2015
#   V2.1.7: Uses Pyfits from Astropy to read FITS files.
#       MC, Oxford, 22 October 2015
#   V2.1.8: Included treatment of the SDSS/MILES vacuum/air wavelength difference.
#       MC, Oxford, 12 August 2016
#   V2.1.9: Automate and test computation of nAge and nMetals.
#       MC, Oxford 1 November 2016
#   V3.0.0: Major upgrade. Compute mass-weighted population parameters and M/L
#       using the new `miles` class which leaves no room for user mistakes.
#       MC, Oxford, 2 December 2016
#   V3.0.1: Make files paths relative to this file, to run the example from
#       any directory. MC, Oxford, 18 January 2017
#   V3.1.0: Use ppxf method pp.plot(gas_component=...) to produce gas emission
#       lines plot. MC, Oxford, 13 March 2017
#   V3.2.0: Uses new ppxf keywords `gas_component` and `gas_names` to print the
#       fluxes and formal errors for the gas emission lines.
#       Uses different kinematic components for the Balmer lines and the rest.
#       MC, Oxford, 28 June 2017
#   V3.3.0: Illustrate how to tie the Balmer emission lines and fit for the
#       gas reddening using the `tie_balmer` keyword. Also limit doublets.
#       MC, Oxford, 31 October 2017
#   V3.3.1: Changed imports for pPXF as a package.
#       Make file paths relative to the pPXF package to be able to run the
#       example unchanged from any directory. MC, Oxford, 17 April 2018
#   V3.3.2: Dropped legacy Python 2.7 support. MC, Oxford, 10 May 2018
#   V4.0.3: Fixed clock DeprecationWarning in Python 3.7.
#       MC, Oxford, 27 September 2018
#
##############################################################################

from time import perf_counter as clock
from os import path

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.miles_util as lib

##############################################################################

def ppxf_example_population_gas_sdss(PATH, name, tie_balmer, limit_doublets):

    ppxf_dir = path.dirname(path.realpath(ppxf_package.__file__))

    # Read SDSS DR8 galaxy spectrum taken from here http://www.sdss3.org/dr8/
    # The spectrum is *already* log rebinned by the SDSS DR8
    # pipeline and log_rebin should not be used in this case.
    #
    #file = ppxf_dir + '/spectra/NGC3522_SDSS_DR8.fits'
    file=PATH+name
    hdu = fits.open(file)
    t = hdu[1].data
    z = float(hdu[2].data.Z) # SDSS redshift estimate

    # Only use the wavelength range in common between galaxy and stellar library.
    #
    mask = (t['loglam'] > np.log10(3540)) & (t['loglam'] < np.log10(7409))
    flux = t['flux'][mask]
    galaxy = flux/np.median(flux)   # Normalize spectrum to avoid numerical issues
    wave = (10**t['loglam'][mask])/(1+z)

    # The SDSS wavelengths are in vacuum, while the MILES ones are in air.
    # For a rigorous treatment, the SDSS vacuum wavelengths should be
    # converted into air wavelengths and the spectra should be resampled.
    # To avoid resampling, given that the wavelength dependence of the
    # correction is very weak, I approximate it with a constant factor.
    #
    wave *= np.median(util.vac_to_air(wave)/wave)

    # The noise level is chosen to give Chi^2/DOF=1 without regularization (REGUL=0).
    # A constant noise is not a bad approximation in the fitted wavelength
    # range and reduces the noise in the fit.
    #
    noise = np.full_like(galaxy, 0.01635)  # Assume constant noise per pixel here

    # The velocity step was already chosen by the SDSS pipeline
    # and we convert it below to km/s
    #
    c = 299792.458  # speed of light in km/s
    velscale = c*np.log(wave[1]/wave[0])  # eq.(8) of Cappellari (2017)
    FWHM_gal = 2.76  # SDSS has an approximate instrumental resolution FWHM of 2.76A.

    #------------------- Setup templates -----------------------

    pathname = ppxf_dir + '/miles_models/Mun1.30*.fits'

    # The templates are not normalized.
    # In this way the weights and mean values are mass-weighted quantities.
    # Use the keyword 'norm_range' for light-weighted quantities.
    miles = lib.miles(pathname, velscale, FWHM_gal)

    # The stellar templates are reshaped below into a 2-dim array with each
    # spectrum as a column, however we save the original array dimensions,
    # which are needed to specify the regularization dimensions
    #
    reg_dim = miles.templates.shape[1:]
    stars_templates = miles.templates.reshape(miles.templates.shape[0], -1)

    # See the pPXF documentation for the keyword REGUL
    regul_err = 0.013  # Desired regularization error

    # Estimate the wavelength fitted range in the rest frame.
    lam_range_gal = np.array([np.min(wave), np.max(wave)])/(1 + z)

    # Construct a set of Gaussian emission line templates.
    # The `emission_lines` function defines the most common lines, but additional
    # lines can be included by editing the function in the file ppxf_util.py.
    gas_templates, gas_names, line_wave = util.emission_lines(
        miles.log_lam_temp, lam_range_gal, FWHM_gal,
        tie_balmer=tie_balmer, limit_doublets=limit_doublets)

    # Combines the stellar and gaseous templates into a single array.
    # During the PPXF fit they will be assigned a different kinematic
    # COMPONENT value
    #
    templates = np.column_stack([stars_templates, gas_templates])

    #-----------------------------------------------------------

    # The galaxy and the template spectra do not have the same starting wavelength.
    # For this reason an extra velocity shift DV has to be applied to the template
    # to fit the galaxy spectrum. We remove this artificial shift by using the
    # keyword VSYST in the call to PPXF below, so that all velocities are
    # measured with respect to DV. This assume the redshift is negligible.
    # In the case of a high-redshift galaxy one should de-redshift its
    # wavelength to the rest frame before using the line below as described
    # in ppxf_example_kinematics_sauron.py and Sec.2.4 of Cappellari (2017)
    #
    c = 299792.458
    dv = c*(miles.log_lam_temp[0] - np.log(wave[0]))  # eq.(8) of Cappellari (2017)
    vel = c*np.log(1 + z)   # eq.(8) of Cappellari (2017)
    start = [vel, 180.]     # (km/s), starting guess for [V, sigma]

    n_temps = stars_templates.shape[1]
    n_forbidden = np.sum(["[" in a for a in gas_names])  # forbidden lines contain "[*]"
    n_balmer = len(gas_names) - n_forbidden

    # Assign component=0 to the stellar templates, component=1 to the Balmer
    # gas emission lines templates and component=2 to the forbidden lines.
    component = [0]*n_temps + [1]*n_balmer + [2]*n_forbidden
    gas_component = np.array(component) > 0  # gas_component=True for gas templates

    # Fit (V, sig, h3, h4) moments=4 for the stars
    # and (V, sig) moments=2 for the two gas kinematic components
    moments = [4, 2, 2]

    # Adopt the same starting value for the stars and the two gas components
    start = [start, start, start]

    # If the Balmer lines are tied one should allow for gas reddeining.
    # The gas_reddening can be different from the stellar one, if both are fitted.
    gas_reddening = 0 if tie_balmer else None

    # Here the actual fit starts.
    #
    # IMPORTANT: Ideally one would like not to use any polynomial in the fit
    # as the continuum shape contains important information on the population.
    # Unfortunately this is often not feasible, due to small calibration
    # uncertainties in the spectral shape. To avoid affecting the line strength of
    # the spectral features, we exclude additive polynomials (degree=-1) and only use
    # multiplicative ones (mdegree=10). This is only recommended for population, not
    # for kinematic extraction, where additive polynomials are always recommended.
    #
    t = clock()
    pp = ppxf(templates, galaxy, noise, velscale, start,
              plot=False, moments=moments, degree=-1, mdegree=10, vsyst=dv,
              lam=wave, clean=False, regul=1./regul_err, reg_dim=reg_dim,
              component=component, gas_component=gas_component,
              gas_names=gas_names, gas_reddening=gas_reddening)

    # When the two Delta Chi^2 below are the same, the solution
    # is the smoothest consistent with the observed spectrum.
    #
    print('Desired Delta Chi^2: %#.4g' % np.sqrt(2*galaxy.size))
    print('Current Delta Chi^2: %#.4g' % ((pp.chi2 - 1)*galaxy.size))
    print('Elapsed time in PPXF: %.2f s' % (clock() - t))

    weights = pp.weights[~gas_component]  # Exclude weights of the gas templates
    weights = weights.reshape(reg_dim)/weights.sum()  # Normalized

    miles.mean_age_metal(weights)
    miles.mass_to_light(weights, band="r")

    # Plot fit results for stars and gas.
    fig=plt.subplots(nrows=1,ncols=2,facecolor='white',dpi=300)
    #plt.clf()
    ax_0=plt.subplot(211)
    ax_0.set_ylim(-2,2)
    pp.plot()
    

    # Plot stellar population mass-fraction distribution
    ax_1=plt.subplot(212)
    miles.plot(weights)
    plt.tight_layout()
    #plt.pause(1)
    
    #fitting the coronal lines and calculate EWs

##############################################################################

In [ ]:
if __name__ == '__main__':
    df=pd.read_csv('/media/richard/Backup Plus/314kpair_DR16_final_Z_0.4_v13_CL_2529_822.csv')
    PATH='/media/richard/Backup Plus/sdss_16_pair/'
    for i in tqdm(df.index):
        name=df.specname[i]
        
        print("\n===============================================\n" +
                 " Fit with free Balmer lines and [SII] doublet: \n" +
                 "===============================================\n")

        ppxf_example_population_gas_sdss(PATH, name, tie_balmer=False, limit_doublets=False)
        plt.savefig('/media/richard/Backup Plus/test0725/%s.png'%df.specname[i])
        print("\n=======================================================\n" +
                 " Fit with tied Balmer lines and limited [SII] doublet: \n" +
                 "=======================================================\n")

        # Note tha the inclusion of a few extra faint Balmer lines is sufficient to
        # decrease the chi2 of the fit, even though the Balmer decrement is fixed.
        # In this case, the best-fitting gas reddening is at the E(B-V)=0 boundary.
        ppxf_example_population_gas_sdss(PATH, name, tie_balmer=True, limit_doublets=True)

In [ ]:
util.emission_lines